<a href="https://colab.research.google.com/github/mamikah/DeepLearning2022Late/blob/main/Selenium%E3%81%A7%E7%94%BB%E5%83%8F%E5%8F%8E%E9%9B%862022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# seleniumを使って自動で画像検索し、結果を抽出する

## 必要なアプリのインストール

### seleniumのインストール（バージョンを指定する）

In [ ]:
!pip install selenium==4.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 958 kB 6.6 MB/s 
     |████████████████████████████████| 384 kB 50.2 MB/s 
     |████████████████████████████████| 140 kB 56.6 MB/s 
     |████████████████████████████████| 4.0 MB 73.5 MB/s 
     |████████████████████████████████| 57 kB 3.0 MB/s 
     |████████████████████████████████| 58 kB 4.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


### chronium-chromedriverのインストール

In [ ]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 7 not upgraded.
Need to get 95.1 MB of archives.
After this operation, 319 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 107.0.5304.87-0ubuntu11.18.04.1 [1,158 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 107.0.5304.87-0ubuntu11.1

## その他のライブラリのimport

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import time
import datetime
# エラー処理用
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## ログ用に時刻を取得

In [ ]:
tm_start = time.time() # 処理時間計測用
dt_now = datetime.datetime.now() # 現在日時
dt_date_str = dt_now.strftime('%Y/%m/%d %H:%M') # 書式指定して変換
print(dt_date_str)

2022/12/05 02:30


## 検索に関する設定値を定義する

In [ ]:
QUERY = '犬 フリー' # 検索したいキーワード
LIMIT_DL_NUM = 100 # ダウンロード数の上限
SAVE_DIR = 'img/dog_' # 出力フォルダ（自動生成させる）
FILE_NAME = 'dog_' # ファイル名（後ろに連番と拡張子を付けていく）
TIMEOUT = 60 # 検索のタイムアウト（60秒）
ACCESS_WAIT = 1 # アクセスする間隔（1秒）←超大事
RETRY_NUM = 3 # クリックやrequestのリトライ回数
DRIVER_PATH = '/user/bin/chromedriver' # chromedriver.exeへのパス

## Cheomeをヘッドレスモードで起動する

ヘッドレスモード = ブラウザを画面に開かないで実行するモード

In [ ]:
#optionの指定を準備
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('start-fullscreen')
options.add_argument('--disable-plugins')
options.add_argument('--disable-extensions')
#起動する
driver = webdriver.Chrome(DRIVER_PATH, options = options)
driver.implicitly_wait(TIMEOUT) #タイムアウトを設定
#ログに処理時間を出力
tm_driver = time.time()
print('WedDriver起動完了', f'{tm_driver - tm_start}s')

<ipython-input-8-52f5ad6b46b4>:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH, options = options)


WebDriverException: ignored

## Googleの画像検索へアクセス

In [ ]:
#QUERY条件付きでURLを定義する
url = f'https://www.google.com/search?q={QUERY}$tdm=isch'
#組み立てたURLでアクセスして検索
driver.get(url)
#時刻のログを収得
tm_geturl = time.time()
print('Google画像検索ページ取得', f'{tm_geturl - tm_driver:.1f}s')

## 検索結果の要素を取得していく

In [ ]:
#ページnサムネイル画像の要素群を取得
#(islmpクラスが指定したimg = サムネイルを全て拾う)
tmb_elems = driver.find_elements_by_css_selector('#islmp img')
#サムネイルのリストから各サムネイルのalt属性を取得していく
tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
#alt属性が空(小さいサムネイル)のものを差し引いて合計を算出
count = len(tmb_alts) - tmb_alts.count('')
print(count)

## 下にスクロールさせてさらに検索結果を表示させてまたカウント

In [ ]:
#定義したLIMIT_DL_NUM(ダウンロード数の上限)になるまでスクロールを繰り返す
while count < LIMIT_DL_NUM:
  #ページの一番下へスクロールして新しいサムネイル画像を表示させる
  driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
  time.sleep(1) #スリープを繰り返す間隔(秒)を入れる←重要
  #(islmpクラスが指定したimg = サムネイルを全て拾う)
  tmb_elems = driver.find_elements_by_css_selector('#islmp img')
  #サムネイルのリストから各サムネイルのalt属性を取得していく
  tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
  #alt属性が空(小さいサムネイル)のものを差し引いて合計を算出
  count = len(tmb_alts) - tmb_alts.count('')
  print(count)
#ループ終わり

## サムネイル画像をクリックすると右側に開かれる領域をオブジェクトに取得

In [ ]:
imgframe_elem = driver.find_element_by_id(('islsp'))

## 画像へURLアクセスして保存する準備をする

In [ ]:
#画像の保存フォルダを作成
os.makedirs(SAVE_DIR, exist_ok = True)
#HTTPリクエスト用のヘッダーを作る
HTTP_HEADERS = {'User-Aget':driver.execute_script('return navigator.userAgent;')}
print(HTTP_HEADERS)
#ダウンロードしていきたい対象のファイル拡張子
IMG_EXTS = ('.jpg', '.jpeg', '.png', '.gif')

## ダウンロード中に利用する関数を定義する
### 画像の拡張子を収得する関数

In [ ]:
from IPython.core import extensions
#拡張子を取得する関数
def get_extension(url):
  url_lower = url.lower()
  for img_ext in IMG_EXTS: #拡張子のリストから1つずつ比較
    if img_ext in url_lower:
      extension = '.jpg' if img_ext == '.jpeg' else img_ext
      break
    else:
      extension = ''
    return extensions

## URLで指定した画像へアクセスし、フォルダ内のファイルとして書き込む

In [ ]:
def download_image(url,path,loop):
  result = False
  for i in range(loop):
    try:
      r = requests.get(url,headers=HTTP_HEADERS,stream=True,timeout = 10)
      r.raise_for_status()
      with open(path,'wb') as f:
        f.write(r.content)
    except requests.exceptions.SSLError:
      print('***** SSL エラー')
      break
    except requests.exceptions.RequestException as e:
      print(f'***** requests エラー（{e}:{i+1}/{RETRY_NUM}）')
      time.sleep(1)
    else:
      result = True

In [ ]:
## サムネイルを取得し、画像ダウンロードの準備ができたタイミング
tm_thumbnails = time.time()
print('サムネイル画像取得とDL準備完了'f'{tm_thumbnails - tm_geturl:.1f}s')

## ダウンロード処理

In [ ]:
EXCLUSION_URL = 'https://lh3.googleusercontent.com/' # 除外対象url
count = 0
url_list = []
# サムネイルをクリック、画像リンクを取得、画像をDL＆保存を繰り返す
# zipで組み合わせの順番を固定化
for tmlelem tmb_alts in zip(tmb_elems,tmb_alts):
  if tmb_alt == '':  # ALT属性が空＝小さいサムネイルとみなしてスキップ
    continue
  print(f'{count}:{tmb_alt}') # カウンタとALT属性を表示
  